In [4]:
from datetime import datetime
import yfinance as yf
import pandas as pd
from backtesting import Backtest, Strategy

import pandas as pd
import talib as ta

pd.set_option('display.float_format', lambda x: '%.6f' % x)

In [5]:
# dji = yf.Ticker("^DJI").history(period="max")
vix = yf.Ticker("^VIX").history(period="max", interval='1h')

In [6]:
from datetime import timedelta
import pytz
import MetaTrader5 as mt5
from datetime import datetime, timedelta


print("MetaTrader5 package author: ", mt5.__author__)
print("MetaTrader5 package version: ", mt5.__version__)

# Establecer conexión con el terminal de MetaTrader 5
if not mt5.initialize():
    raise Exception("initialize() failed, error code =", mt5.last_error())

# Establecer la zona horaria a UTC
timezone = pytz.timezone("Etc/UTC")

# Crear objetos 'datetime' en zona horaria UTC
now = datetime.now(tz=timezone)
date_from = now - timedelta(days=20) - timedelta(days=500) 
            
# Obtener las tasas históricas
rates = mt5.copy_rates_range('USTECm', mt5.TIMEFRAME_H1, date_from, now)

# Crear DataFrame con las tasas
df = pd.DataFrame(rates)

# Convertir el tiempo de segundos a formato datetime
df['time'] = pd.to_datetime(df['time'], unit='s')


# Renombrar columnas para el ticker principal
df = df.rename(columns={
    'time': 'Date',
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'tick_volume': 'Volume'
}).set_index('Date')


# Cerrar la conexión con MetaTrader 5
mt5.shutdown()

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288


True

In [7]:
vix.rename(
    columns={'Close':'VixClose'}, inplace=True
)

In [8]:
df.index = df.index.tz_localize('UTC').tz_convert('UTC')
vix.index = vix.index.tz_convert('UTC')

In [9]:
merge = pd.merge(
    df, 
    vix[['VixClose']], 
    left_index=True, 
    right_index=True
)

merge

,Open,High,Low,Close,Volume,spread,real_volume,VixClose
2023-04-12 12:00:00+00:00,12991.310000,13138.870000,12950.000000,13073.070000,3433,361,0,19.370001
2023-04-12 13:00:00+00:00,13074.100000,13090.100000,12931.840000,12964.060000,4527,378,0,18.620001
2023-04-12 14:00:00+00:00,12962.170000,13025.820000,12904.750000,12907.540000,4710,383,0,18.610001
2023-04-12 15:00:00+00:00,12905.330000,12976.190000,12874.650000,12969.660000,4006,358,0,19.070000
2023-04-12 16:00:00+00:00,12969.750000,12988.890000,12962.480000,12983.720000,3431,352,0,18.840000
...,...,...,...,...,...,...,...,...
2024-09-12 20:00:00+00:00,19442.130000,19448.840000,19392.650000,19404.070000,2767,510,0,17.070000
2024-09-13 07:00:00+00:00,19431.160000,19469.280000,19425.850000,19426.930000,2763,505,0,17.110001
2024-09-13 08:00:00+00:00,19427.830000,19458.990000,19417.550000,19445.320000,2494,504,0,17.020000
2024-09-13 09:00:00+00:00,19445.310000,19471.070000,19441.180000,19454.880000,2270,503,0,16.889999


In [10]:
def ll_hh_indicator(close, window=None):
    if type(close) != pd.Series:
        close = close.s

    rolling_min = close.rolling(window=window, min_periods=1).min()
    is_lower_low = close == rolling_min

    rolling_max = close.rolling(window=window, min_periods=1).max()
    is_higher_high = close == rolling_max

    return is_lower_low, is_higher_high


class VixRsi(Strategy):

    def init(self):
        self.rsi = self.I(ta.RSI, self.data.Close, timeperiod=2)
        self.sma = self.I(ta.SMA, self.data.Close, timeperiod=200)

        self.vix_sma = self.I(ta.SMA, self.data.VixClose, timeperiod=10)
        self.lower_low, self.higher_high = self.I(ll_hh_indicator, self.data.Close, window=5)


    def next(self):
        actual_close = self.data.Close[-1]

        if self.position:
            first_trade = self.trades[0]
            today = self.data.index[-1].tz_convert('UTC')
            time_in_position = (today - first_trade.entry_time.tz_convert('UTC'))
            time_in_position = time_in_position.days

            if self.position.is_long:
                if self.higher_high:
                    self.position.close()

        else:
            cum_rsi = self.rsi[-1] + self.rsi[-2]

            vix_sma_value = self.vix_sma[-1]
            vix_close = self.data.VixClose[-1]
            vix_above_sma = vix_close > (vix_sma_value * (1 + 0.05))

            if vix_above_sma and cum_rsi <= 50 and actual_close > self.sma:
                self.buy(size=1)


In [11]:
# start_date = '2022-01-01'
# end_date = '2022-02-01'


bt = Backtest(
    merge, 
    VixRsi, 
    cash=100000, 
    commission=7e-4
)

stats = bt.run()


stats

Start                     2023-04-12 12:00...
End                       2024-09-13 10:00...
Duration                    519 days 22:00:00
Exposure Time [%]                    3.042434
Equity Final [$]                100607.399181
Equity Peak [$]                 100616.565196
Return [%]                           0.607399
Buy & Hold Return [%]               48.862968
Return (Ann.) [%]                    0.413288
Volatility (Ann.) [%]                0.383441
Sharpe Ratio                         1.077842
Sortino Ratio                        1.922337
Calmar Ratio                         1.476490
Max. Drawdown [%]                   -0.279913
Avg. Drawdown [%]                   -0.088728
Max. Drawdown Duration      164 days 02:00:00
Avg. Drawdown Duration       36 days 01:00:00
# Trades                                   23
Win Rate [%]                        82.608696
Best Trade [%]                       1.194868
Worst Trade [%]                     -1.054048
Avg. Trade [%]                    

In [12]:
bt.plot()

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p1747', ...)

In [13]:
stats, heatmap = bt.optimize(
    upper_bound=range(50, 85, 5),
    lower_bound=range(10, 45, 5),
    rsi_window=range(10, 20, 2),
    maximize='Sharpe Ratio',
    # maximize=optim_func,
    # max_tries=100,
    return_heatmap=True
)

  0%|          | 0/9 [00:00<?, ?it/s]

AttributeError: Strategy 'VixRsi' is missing parameter 'upper_bound'.Strategy class should define parameters as class variables before they can be optimized or run with.